In [1]:
import pandas as pd
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
import numpy as np

In [5]:
from sklearn.model_selection import RandomizedSearchCV

In [6]:
from sklearn.metrics import accuracy_score

# # Train an XGBoost model
# model = XGBClassifier(num_class=6, objective="multi:softmax", eval_metric=["mlogloss"], random_state=42)
# model.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(X_test)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)

# # Print accuracy
# print(f"Accuracy: {accuracy:.2f}")

from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier(
    loss="log_loss",  # Best for multi-class classification
    early_stopping=True,  # Stops training when performance stops improving
    random_state=42
)
model.fit(X_train, y_train)


# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print accuracy
print(f"Accuracy: {accuracy:.2f}")


c:\Users\User\Desktop\Shehara\PROJECTS\Emotion detector\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.58


In [7]:
params = {
    'max_iter': np.arange(100, 1000, 100),  # Controls boosting rounds
    'learning_rate': np.linspace(0.001, 0.1, 10),  # Step size for optimization
    'max_depth': np.arange(4, 12, 2),  # Controls tree depth
    'min_samples_leaf': np.arange(5, 50, 5),  # Prevents overfitting
    'l2_regularization': np.linspace(0, 1, 5),  # Regularization to control model complexity
    'max_bins': np.arange(128, 256, 32)  # Helps with numerical feature binning
}

In [8]:
# Perform Randomized Search
random_search = RandomizedSearchCV(
    model,
    params,
    cv=10,  # 10-fold cross-validation for stability
    scoring='accuracy',
    n_iter=20,  # Number of random combinations to try
    n_jobs=-1,  # Use all CPU cores
    verbose=2,  # Shows progress updates
    random_state=42
)

In [9]:
# Fit on training data
random_search.fit(X_train, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


c:\Users\User\Desktop\Shehara\PROJECTS\Emotion detector\.venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=10,
                   estimator=HistGradientBoostingClassifier(early_stopping=True,
                                                            random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'l2_regularization': array([0.  , 0.25, 0.5 , 0.75, 1.  ]),
                                        'learning_rate': array([0.001, 0.012, 0.023, 0.034, 0.045, 0.056, 0.067, 0.078, 0.089,
       0.1  ]),
                                        'max_bins': array([128, 160, 192, 224]),
                                        'max_depth': array([ 4,  6,  8, 10]),
                                        'max_iter': array([100, 200, 300, 400, 500, 600, 700, 800, 900]),
                                        'min_samples_leaf': array([ 5, 10, 15, 20, 25, 30, 35, 40, 45])},
                   random_state=42, scoring='accuracy', verbose=2)

In [10]:
# Print best parameters
print("Best Parameters:", random_search.best_params_)

# Get the best model
best_model = random_search.best_estimator_

Best Parameters: {'min_samples_leaf': np.int64(45), 'max_iter': np.int64(900), 'max_depth': np.int64(10), 'max_bins': np.int64(160), 'learning_rate': np.float64(0.067), 'l2_regularization': np.float64(1.0)}


In [11]:
# Evaluate on test data (optional)
accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.591116173120729


In [14]:
import joblib
joblib.dump(model, 'histgradientmodel.pkl')

['histgradientmodel.pkl']